In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import keras
import imutils
from keras.models import load_model
from keras.models import model_from_json
from keras.preprocessing.image import img_to_array
%matplotlib inline
from textgenrnn import textgenrnn
from multiprocessing import Process,Queue
from multiprocessing import Pool

Using TensorFlow backend.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [2]:
model=load_model("./models/hands_sign_binary_final.hdf5")
txt=textgenrnn()

In [3]:
def edgedetect( channel):
    sobelX=cv2.Sobel(channel,cv2.CV_16S,1,0)
    sobelY=cv2.Sobel(channel,cv2.CV_16S,0,1)
    sobel=np.hypot(sobelX,sobelY)
    sobel[sobel>255]=255
    return sobel


def findSignificantContours (img, edgeImg):
    contours, heirarchy = cv2.findContours(edgeImg, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    level1 = []
    for i, tupl in enumerate(heirarchy[0]):
        # Each array is in format (Next, Prev, First child, Parent)
        # Filter the ones without parent
        if tupl[3] == -1:
            tupl = np.insert(tupl, 0, [i])
            level1.append(tupl)
    significant = []
    tooSmall = edgeImg.size * 5 / 100 # If contour isn't covering 5% of total area of image then it probably is too small
    for tupl in level1:
        contour = contours[tupl[0]]
        area = cv2.contourArea(contour)
        if area > tooSmall:
            significant.append([contour, area])
            # Draw the contour on the original image
            #cv2.drawContours(img, [contour], 0, (0,255,0),2, cv2.LINE_AA, maxLevel=1)
            #print("______________________________________")
    significant.sort(key=lambda x: x[1])
    #print ([x[1] for x in significant]);
    return [x[0] for x in significant]

cap_region_x_begin=0.5  # start point/total width
cap_region_y_end=0.8  # start point/total width
threshold = 60  #  BINARY threshold
blurValue = 41  # GaussianBlur parameter
bgSubThreshold = 100
learningRate = 0
var_list=[]
# variables
isBgCaptured = 0   # bool, whether the background captured
triggerSwitch = False  # if true, keyborad simulator works

def printThreshold(thr):
    print("! Changed threshold to "+str(thr))


def removeBG(frame):
    fgmask = bgModel.apply(frame,learningRate=learningRate)
    # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    # res = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    kernel = np.ones((3, 3), np.uint8)
    fgmask = cv2.erode(fgmask, kernel, iterations=1)
    res = cv2.bitwise_and(frame, frame, mask=fgmask)
    return res

def detect(gray,frame,thresh):
    cv2.rectangle(frame,(520,100),(790,500),(255,0,0),0)
    #img_roi=frame[70:500,520:890]
    #img_roi=frame[100:500,520:790]
    lab={'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'k': 9, 'l': 10, 'm': 11, 'n': 12, 'o': 13, 'p': 14,
     'q': 15, 'r': 16, 's': 17, 't': 18, 'u': 19, 'v': 20, 'w': 21, 'x': 22, 'y': 23}
    global var_list
    """
    # step 1 blurring
    blurred=cv2.GaussianBlur(img_roi,(5,5),0)
    # Step 2 Sobel Detection:- As we are dealing with 3 colors we need it for all colors
    edgeImg = np.max(np.array([edgedetect(blurred[:,:, 0]), edgedetect(blurred[:,:, 1]), edgedetect(blurred[:,:, 2]) ]), axis=0)
    #edgeImg = np.max(np.array([edgedetect(blurred[:,:, 0])]), axis=0)
    # Step 3 Removal of noise
    mean=np.mean(edgeImg)
    edgeImg[edgeImg<=mean]=0
    cv2.imshow("edge",edgeImg)
    
    #Step 4 Contour Detection and areal detection
    edgeImg_8u = np.asarray(edgeImg, np.uint8)
    cv2.imshow("edge",edgeImg_8u)
    
    # Find contours
    significant = findSignificantContours(img_roi, edgeImg_8u)
    
    # Mask
    mask = edgeImg.copy()
    mask[mask > 0] = 0
    
    cv2.fillPoly(mask, significant, 200)
    
    # Invert mask
    mask = np.logical_not(mask)
    
    #print(mask)
    #Finally remove the background
    img_roi[mask] = 0
    """
    #img_roi=fgbg.apply(img_roi)
    #cv2.imshow("imag",img_roi)
    #print(img_roi.shape)
    #image=cv2.resize(img_roi,(28,28))
    
    image=cv2.resize(thresh,(28,28))
    image=image.astype("float")/255.0
    image=img_to_array(image)
    image=np.expand_dims(image,axis=0)
    pred=model.predict(image)
    prob=np.max(pred)
    labels=np.argmax(pred)
    for i,j in lab.items():
        if j==labels:
            labels=i
    
    if prob>0.8:
        cv2.putText(frame,labels,(500,50),cv2.FONT_HERSHEY_COMPLEX,2,(255,0,0),2,cv2.LINE_AA,False)
        #a=txt.generate(n=3,return_as_list=True,max_gen_length=5,prefix=labels)
        #print(a)
        #b=int(input("Which One Do You Like To Choose?"))
        #b=b-1
        #cv2.putText(frame,a[b],(500,50),cv2.FONT_HERSHEY_COMPLEX,2,(255,0,0),2,cv2.LINE_AA,False)
        var_list.append(labels)
        word_gen(var_list)
        print("labels:",labels,"probability:",prob)
        
    #cv2.imshow("roi",img_roi)
    return frame

def word_gen(var_list):
    if len(var_list)>0:
        a=txt.generate(n=1,return_as_list=True,max_gen_length=4,prefix=var_list[-1])
        cv2.putText(frame,a[-1],(500,50),cv2.FONT_HERSHEY_COMPLEX,2,(255,0,0),2,cv2.LINE_AA,False)

Here I show you how to do segmentation for “simple” images like these. The algorithm that I have used is as follows:

1. Run an edge detection algorithm on the image (like Sobel, Scharr or Prewitt)
2. Reduce noise on the resulting edge image (using a simple trick I found from Octave forge/Matlab)
3. Run contour detection over the edges, return the contour in hierarchical order and pick the contours in the first level heirarchy.
4. Now either pick the largest contour (in this case) or ignore all contours whose area is less than 5% of the total image area.


In [4]:
cap=cv2.VideoCapture(-1)
#fgbg=cv2.createBackgroundSubtractorMOG2()
cap.set(10,200)
cv2.namedWindow('trackbar')
cv2.createTrackbar('trh1', 'trackbar', threshold, 100, printThreshold)
count=0

while True:
    ret,frame=cap.read()
    threshold = cv2.getTrackbarPos('trh1', 'trackbar')
    frame = cv2.bilateralFilter(frame, 5, 50, 100)  # smoothing filter
    frame=cv2.resize(frame,(940,640),interpolation=cv2.INTER_LINEAR)
    frame=cv2.flip(frame,1)
    frame1=frame[100:500,520:790]
    if isBgCaptured == 1:  # this part wont run until background captured
        img = removeBG(frame1)
        #cv2.imshow('mask', img)
        # convert the image into binary image
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (blurValue, blurValue), 0)
        #cv2.imshow('blur', blur)
        ret, thresh = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY)
        cv2.imshow('roi', thresh)
        #gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        canvas=detect(gray,frame,thresh)
        cv2.imshow("image",canvas)
        
    k = cv2.waitKey(10)
    if k == ord('q'):  # press ESC to exit
        cap.release()
        cv2.destroyAllWindows()
        break
    elif k == ord('b'):  # press 'b' to capture the background
        bgModel = cv2.createBackgroundSubtractorMOG2(0, bgSubThreshold)
        isBgCaptured = 1
        print( '!!!Background Captured!!!')
    elif k == ord('r'):  # press 'r' to reset the background
        bgModel = None
        triggerSwitch = False
        isBgCaptured = 0
        print ('!!!Reset BackGround!!!')

cap.release()
cv2.destroyAllWindows()


!!!Background Captured!!!
labels: h probability: 0.9861828
labels: h probability: 0.9868846
labels: h probability: 0.98555005
labels: h probability: 0.9887318
labels: h probability: 0.98701197
labels: h probability: 0.8361767
labels: r probability: 0.9473642
labels: m probability: 0.97171086
labels: m probability: 0.8230799
labels: m probability: 0.9001677
labels: m probability: 0.9243826
labels: m probability: 0.9556321
labels: m probability: 0.8896357
labels: m probability: 0.88181174
labels: m probability: 0.92271286
labels: m probability: 0.9349778
labels: m probability: 0.9542408
labels: m probability: 0.82290155
labels: m probability: 0.8398632
labels: m probability: 0.856513
labels: m probability: 0.8793482
labels: m probability: 0.85866714
labels: m probability: 0.93967664
labels: m probability: 0.9065221
labels: m probability: 0.93306035
labels: m probability: 0.92699593
labels: h probability: 0.8474716
labels: h probability: 0.8432636
labels: b probability: 0.9016194
labels: 

labels: h probability: 0.8559302
labels: h probability: 0.9275067
labels: h probability: 0.96099675
labels: h probability: 0.9758858
labels: h probability: 0.9354593
labels: h probability: 0.9348936
labels: h probability: 0.95414597
labels: h probability: 0.9723256
labels: h probability: 0.9971674
labels: h probability: 0.8651642
labels: h probability: 0.9928833
labels: h probability: 0.9997917
labels: h probability: 0.92674446
labels: h probability: 0.9022165
labels: h probability: 0.92809826
labels: b probability: 0.9827145
labels: b probability: 0.97990316
labels: b probability: 0.9892315
labels: h probability: 0.95295763
labels: b probability: 0.996326
labels: h probability: 0.997982
labels: h probability: 0.9528398
labels: h probability: 0.94491667
labels: h probability: 0.9816172
labels: h probability: 0.99207205
labels: h probability: 0.93642503
labels: h probability: 0.8869821
labels: h probability: 0.929965
labels: p probability: 0.80973744
labels: p probability: 0.96525705
la

In [6]:
def check(k,dirname):
    count=0
    image=cv2.imread(k)
    image=image.astype("float")/255.0
    image=img_to_array(image)
    image=np.expand_dims(image,axis=0)
    pred=model.predict(image)
    prob=np.max(pred)
    labels=np.argmax(pred)
    lab={'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'k': 9, 'l': 10, 'm': 11, 'n': 12, 'o': 13, 'p': 14,
     'q': 15, 'r': 16, 's': 17, 't': 18, 'u': 19, 'v': 20, 'w': 21, 'x': 22, 'y': 23}
    for i,j in lab.items():
        if j==labels:
            labels=i
    #labels=chr(65+labels)
    if labels!=dirname:
        count+=1
    #print(prob,labels)
    return count

In [5]:
import os
count=0
for i in os.listdir("./handsign_images1/cool_test/"): 
    for j in os.listdir("./handsign_images1/cool_test/"+i+"/"):
        a=check("./handsign_images1/cool_test/"+i+"/"+j,i)
        print(count)
        if a==1:
            count+=1
print(count)

0
0
0
1
1
2
2
2
3
3
4
4
4
4
4
5
5
5
5
5
6
6
6
6
6
6
6
6
7
7
7
7
7
7
7
7
7
7
7
7
7
8
9
9
9
9
9
9
10
10
11
11
11
11
11
11
11
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
12
13
14
14
14
15
15
15
15
15
15
15
15
16
16
17
17
17
18
18
18
18
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
19
20
20
21
22
22
22
22
22
22
22
22
22
22
22
22
23
23
23
23
23
23
23
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
24
25
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
26
27
27
27
27
28
28
28
28
29
29
29
30
31
31
31
31
31
31
31
31
32
32
32
32
32
32
33
33
33
33
34
35
36
36
36
36
36
36
36
36
36
36
36
37
37
37
37
37
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
39
40
40
40
40
40
40
40
40
40
40
40
40
40
40
41
41
41
41
41
41
41
41
41
41
42
42
42
42
42
42
43
43
43
43
43
43
43
43
43
43
43
43
43
43
44
44
44
44
44
44
44
45
45
45
45
45
45
46
47
47
47
47
47
48
48
48
48
48
48
48
48
48
48
48
48
48
49
49
49
49
49
49
49
49
49
49
50
50
51
51
51
52
52
5

374
374
374
374
374
374
374
374
374
374
374
374
374
375
375
375
376
376
376
376
376
376
376
376
376
376
376
377
377
377
377
377
377
377
377
377
377
377
377
377
377
377
377
377
377
377
377
377
377
377
378
378
378
378
378
378
379
379
380
380
381
382
383
383
383
383
383
384
385
385
385
386
386
386
386
387
388
388
388
388
388
388
388
388
388
388
388
388
388
388
388
389
389
389
389
389
389
389
389
389
389
389
389
389
390
390
390
390
390
390
390
390
391
392
392
392
392
392
392
392
393
394
394
394
394
394
394
394
394
394
394
394
394
394
395
395
395
396
396
396
396
396
397
398
398
398
398
398
398
398
398
399
399
399
399
399
399
400
400
400
400
401
401
401
401
401
401
401
401
401
402
402
402
402
402
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
403
404
405
406
406
407
407
407
407
407
407
407
407
407
407
407
407
407
407
407
407
407
407
407
407
407
408
409
409
409
409
409
410
410
411
411
411
411
412
413
413
413
414
414
415
415
415
415
415


717
717
718
718
718
718
719
719
720
720
720
721
722
723
723
724
724
725
725
726
726
726
727
727
727
728
728
728
728
729
729
730
731
732
732
732
732
733
734
735
735
736
737
738
739
740
741
742
742
742
742
742
742
742
742
743
743
744
744
744
744
745
746
746
747
747
748
749
749
750
750
751
751
751
752
753
753
753
753
754
755
755
755
756
757
757
757
757
757
757
758
758
759
759
759
760
760
760
760
761
761
762
762
763
763
764
764
765
766
767
767
768
768
769
770
771
772
772
773
773
773
773
774
774
774
774
774
775
775
775
775
776
776
776
776
776
777
778
778
779
779
780
781
781
782
783
783
784
785
785
786
786
786
786
787
787
787
788
788
788
788
789
790
791
792
793
794
795
796
796
796
796
796
796
797
797
798
799
799
799
799
799
799
799
800
801
801
802
802
802
802
802
803
803
804
805
805
805
806
806
806
807
807
807
807
808
809
810
811
812
813
813
813
813
813
814
814
815
815
816
816
817
817
817
817
817
817
818
818
819
819
819
819
820
821
821
822
822
822
822
823
824
825
825
825
825
825
825
825
825


984
984
984
985
985
985
985
985
985
985
985
985
985
985
985
985
985
985
985
986
986
986
986
986
987
987
987
987
987
987
987
988
988
988
988
988
988
988
988
988
988
988
988
988
988
989
989
990
990
990
990
990
990
991
991
992
992
992
992
992
992
992
992
992
992
992
992
992
992
993
993
994
995
996
996
996
996
996
996
996
996
997
998
998
998
998
998
998
999
999
999
999
999
999
999
999
999
999
999
1000
1000
1000
1001
1001
1001
1001
1001
1001
1001
1001
1001
1002
1002
1002
1002
1002
1002
1002
1003
1003
1003
1003
1004
1005
1005
1005
1005
1005
1006
1006
1006
1006
1006
1006
1006
1007
1007
1007
1007
1007
1007
1007
1007
1007
1008
1008
1009
1009
1010
1011
1011
1012
1012
1012
1012
1012
1013
1013
1013
1013
1013
1013
1013
1014
1014
1014
1014
1015
1015
1016
1016
1017
1017
1018
1018
1018
1018
1018
1018
1018
1019
1019
1019
1019
1019
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1020
1021
1021
1022
1022
1023
1023
1023
1023
1023
1023
1023
1023
1023
1023
1024
1024

In [3]:
import cv2
import numpy as np

cap=cv2.VideoCapture(0)
fgbg=cv2.createBackgroundSubtractorMOG2()

while True:
    ret,frame=cap.read()
    frame=cv2.flip(frame,1)
    fgmask=fgbg.apply(frame)
    cv2.imshow("frame",frame)
    cv2.imshow("mask",fgmask)
    if cv2.waitKey(1)&0xff==ord('q'):
        cv2.destroyAllWindows()
        break
cap.release()
cv2.destroyAllWindows()

In [2]:
import numpy as np

In [13]:
model.input_shape

(None, 28, 28, 3)

In [ ]:
txt=textgenrnn()
txt.generate(interactive=True,top_n=5,prefix="a")


Controls:
	s: stop.	x: backspace.	o: write your own.

Options:
	1:  
	2: n
	3: l
	4: t
	5: b

Progress: a

Your choice?
> 2
Controls:
	s: stop.	x: backspace.	o: write your own.

Options:
	1: y
	2: d
	3:  
	4: n
	5: i

Progress: an

Your choice?
> s
Controls:
	s: stop.	x: backspace.	o: write your own.

Options:
	1: A
	2: O
	3: E
	4: I
	5: o

Progress: an<s>

Your choice?
> end
That's not an option!
Controls:
	s: stop.	x: backspace.	o: write your own.

Options:
	1: A
	2: O
	3: E
	4: I
	5: o

Progress: an<s>

Your choice?
> end
That's not an option!
Controls:
	s: stop.	x: backspace.	o: write your own.

Options:
	1: A
	2: O
	3: E
	4: I
	5: o

Progress: an<s>

Your choice?
> .
That's not an option!
Controls:
	s: stop.	x: backspace.	o: write your own.

Options:
	1: A
	2: O
	3: E
	4: I
	5: o

Progress: an<s>

Your choice?
> >
That's not an option!
Controls:
	s: stop.	x: backspace.	o: write your own.

Options:
	1: A
	2: O
	3: E
	4: I
	5: o

Progress: an<s>

Your choice?
> <
That's not an opti

In [1]:
import  numpy as np

In [ ]:
np.expand_dims()